In [1]:
import json
import numpy as np
from __future__ import print_function
import codecs
from textrank4zh import TextRank4Keyword, TextRank4Sentence
tr4w = TextRank4Keyword()
tr4s = TextRank4Sentence()

## Read Data

In [2]:
with open('data/kp.json', encoding='utf-8') as f:
    data_kp = json.load(f)
with open('data/yao.json', encoding='utf-8') as f:
    data_yao = json.load(f)

In [4]:
print(len(data_kp['data']))
print(len(data_yao['data']))

1545
1027


## Posts and comments summarization

In [5]:
def summarization(message, num_sum_sent):
    if len(message.split('。')) or len(message.split(' '))> num_sum_sent:
        sum_message = ''
        tr4s.analyze(text=message, lower=True, source = 'all_filters')
        for item in tr4s.get_key_sentences(num=num_sum_sent):
            sum_message = sum_message + item.sentence + '。'
    else:
        sum_message = message
    return sum_message

In [15]:
def processing(data):
    sum_post_message_list = []
    post_time_list = []
    sum_comment_message_list = []
    comment_time_list = []
    comments_to_post_index = []
    idx = 0
    post_idx = 0
    for post in data['data']:
        print(str(idx) + '/' + str(len(data['data'])))
        idx = idx + 1
        if 'message' in post.keys():
            #posts
            post_message = post['message'].replace("\n", "")
            post_time = post['created_time']
            sum_post_message = summarization(post_message, num_sum_sent=10)
            sum_post_message_list.append(sum_post_message)
            post_time_list.append(post_time)

            #comments
            if 'comments' in post.keys():
                comments_list = post['comments']
                for comment in comments_list:
                    comment_message = comment['message'].replace("\n", "")
                    comment_time = comment['created_time']
                    sum_comment_message = summarization(comment_message, num_sum_sent=3)
                    sum_comment_message_list.append(sum_comment_message)
                    comments_to_post_index.append(post_idx)
                    comment_time_list.append(comment_time)
            post_idx = post_idx + 1
            
    print('processing done')
    return sum_post_message_list, post_time_list, sum_comment_message_list, comment_time_list, comments_to_post_index


In [16]:
kp_sum_posts, kp_posts_time, kp_sum_comments, kp_comments_time, kp_comments_to_post_index = processing(data_kp)
yao_sum_posts, yao_posts_time, yao_sum_comments, yao_comments_time, yao_comments_to_post_index  = processing(data_yao)

processing done
processing done


In [22]:
from sklearn.externals import joblib
joblib.dump((kp_sum_posts,kp_posts_time, kp_sum_comments, kp_comments_time, kp_comments_to_post_index), "result/kp_all.pkl")
joblib.dump((yao_sum_posts,yao_posts_time, yao_sum_comments, yao_comments_time, yao_comments_to_post_index), "result/yao_all.pkl")

['np/yao_all.pkl']